Project 2: Messier Object Tourist Guide
Astronomy 1221 — Project Suggestion #2: Data Wrangling and Analysis
Authors: Torsten Wigglesworth and Dilmar Roblero
Pathway: A (LLM + Function Tools + Pandas)  
Description: An AI-powered observing companion that organizes Messier catalog data by season, brightness, and object type, generating personalized observing guides. 

An AI powered messier object guide. This project focused on taking a Messier CSV on all the Messier catalog and manipulating the data into pandas dataframes. These dataframes are used to find out all sorts of information on the Messier system such as their constellations, best viewing time, and more. This information is given and asked for in natural language by using an LLM called Claude which will answer any questions on the Messier system. Claude will use the built in functions first to find any information and if that's not enough it will find the answer on its own.

All necessary imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, re
import anthropic
from dotenv import load_dotenv
import json

This fills a pandas data frame with the Messier catalog. 

In [2]:

#Reads through the provided messier csv and fills data frames which will later be used to find out information on all the different messier objects.
df = pd.read_csv("Messier.csv", encoding="latin1", engine="python", skiprows=17).dropna(axis=1, how="all")
ALIASES = {
    "Name": ["Name","Messier","Object","M"],
    "Constellation": ["Constellation","Constellation_name"],
    "Class": ["Class","Type","Object_type"],
    "Magnitude": ["Magnitude","Mag","Vmag","Apparent_magnitude"],
    "Right_ascension": ["Right_ascension","RA","Right Ascension","RA_hms"],
    "Declination": ["Declination","Dec","DEC","Declination_deg"],
    "Remarks": ["Remarks","Notes","Description","Comment"],
}
#reformats all the data into something easier to parse through
dfmod = {c.lower().replace(" ","_"): c for c in df.columns}
def pick(colnames):
    for c in colnames:
        if c in df.columns:
            return c
    for c in colnames:
        k = c.lower().replace(" ","_")
        if k in dfmod:
            return dfmod[k]
    return None

DF = pd.DataFrame({k: df[pick(v)] if pick(v) is not None else np.nan for k,v in ALIASES.items()})

MONTHS = ["Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec","Jan","Feb"]
def ra_to_month(ra_h):
    if pd.isna(ra_h): return None
    return MONTHS[int(((ra_h + 12) % 24) / 2)]

DF["RA_hours"] = pd.to_numeric(DF["Right_ascension"], errors="coerce")
DF["Dec_deg"] = pd.to_numeric(DF["Declination"], errors="coerce")
DF["Best_Month"] = DF["RA_hours"].apply(ra_to_month)
DF["Class"] = DF["Class"].astype(str)
DF["Magnitude"] = pd.to_numeric(DF["Magnitude"], errors="coerce")

LLM function tools. These tools will be used by the LLM to answer specific questions related to their functions

In [3]:
#Goes through each data point which relates to each function and adds them to the dictionary for later use. 
def get_by_season(month):
    m = month[:3].capitalize()
    res = DF[DF["Best_Month"] == m]
    res = res.sort_values(["Magnitude", "Name"], na_position = "last")
    return res[["Name", "Constellation", "Class", "Magnitude", "Best_Month"]].to_dict(orient="records")
#Finds the different types of each of the messier objects
def filter_by_type(object_type):
    mask = DF["Class"].str.contains(object_type, case = False, na = False)
    res = DF[mask].sort_values(["Magnitude", "Name"], na_position="last")
    return res[["Name", "Constellation", "Class", "Magnitude", "Best_Month"]].to_dict(orient="records")
#Finds the magnitude of each object
def find_by_magnitude(min_mag, max_mag):
    mask = (DF["Magnitude"] >= min_mag) & (DF["Magnitude"] <= max_mag)
    res = DF[mask].sort_values(["Magnitude", "Name"], na_position="last")
    return res[["Name", "Class", "Magnitude", "Best_Month"]].to_dict(orient="records")
#adds all their information to a story about each object to easility access later.
def get_object_story(name):
    match = DF.loc[DF["Name"].str.lower() == name.lower()]
    if match.empty:
        return {"error": "Object not found."}
    r = match.iloc[0]
    story = f"{r['Name']}"
    if pd.notna(r.Constellation):
        story += f" ({r.Constellation})"
    if pd.notna(r.Class):
        story += f"; {r.Class}"
    if pd.notna(r.Magnitude):
        story += f"; mag {r.Magnitude:.1f}"
    if pd.notna(r.RA_hours) and pd.notna(r.Dec_deg):
        story += f"; RA {r.RA_hours:.2f}h, Dec {r.Dec_deg:.1f}°"
    if pd.notna(r.Best_Month):
        story += f"; Best month: {r.Best_Month}"
    if pd.notna(r.Remarks):
        story += f"; Notes: {r.Remarks}"
    return {"name": r.Name, "story": story}


This schema is used by the LLM to know which functions it has access to.

In [4]:
#schema of the required tools for the LLM
TOOLS = [
    {
        "name": "get_by_season",
        "description": "Return objects best viewed in a given month (e.g., Apr).",
        "input_schema": {"type": "object", "properties": {"month": {"type": "string"}}, "required": ["month"]},
    },
    {
        "name": "filter_by_type",
        "description": "Return objects matching a type (galaxy, nebula, cluster).",
        "input_schema": {"type": "object", "properties": {"object_type": {"type": "string"}}, "required": ["object_type"]},
    },
    {
        "name": "find_by_magnitude",
        "description": "Return objects within a magnitude range (inclusive).",
        "input_schema": {"type": "object", "properties": {"min_mag": {"type": "number"}, "max_mag": {"type": "number"}}, "required": ["min_mag", "max_mag"]},
    },
    {
        "name": "get_object_story",
        "description": "Return a one-paragraph story for a Messier object by name.",
        "input_schema": {"type": "object", "properties": {"name": {"type": "string"}}, "required": ["name"]},
    },
]


LLM Connection and Prompts

In [5]:
#Fully sets out the LLM through the anthropic key and prompts the LLM to be specifically helpful with messier objects and to use the built in functions
#when possible
load_dotenv("config.env")
CLAUDE_MODEL = "claude-3-5-haiku-latest"
client = anthropic.Anthropic()
MODEL = os.getenv("CLAUDE_MODEL", "claude-3-5-haiku-latest")
SYSTEM = (
    "You are a concise Messier assistant. "
    "Use EXACTLY these four tools when helpful: get_by_season, filter_by_type, find_by_magnitude, get_object_story."
    "Answer questions about the messier catalog when prompted"
    "If user says just quit(regardless of capitilization) respond with just the words Goodbye"
)

print("Welcome to the Messier Object Tourist Guide!")
print("You can ask for a variety of information all about Messier Objects such as their type, magnitude, best viewing time, etc.")
print("Quit → stop the program\n")
#Main loop which takes in user input and prompts the LLM for the information
while True:
        user_text = input("\n> ").strip()
        if not user_text:
            continue
            
        first = client.messages.create(
            model=MODEL,
            tools=TOOLS,
            max_tokens=600,
            temperature=0.2,
            system=SYSTEM,
            messages=[{"role": "user", "content": user_text}],
        )

        output_text = ""

        for part in first.content:
            if getattr(part, "type", None) == "tool_use":
                name = part.name
                args = part.input or {}

                if name == "get_by_season":
                    result = get_by_season(args["month"])
                elif name == "filter_by_type":
                    result = filter_by_type(args["object_type"])
                elif name == "find_by_magnitude":
                    result = find_by_magnitude(args["min_mag"], args["max_mag"])
                elif name == "get_object_story":
                    result = get_object_story(args["name"])
                else:
                    result = {"error": f"unknown tool {name}"}

                follow = client.messages.create(
                    model=MODEL,
                    tools=TOOLS,
                    max_tokens=600,
                    temperature=0.2,
                    system=SYSTEM,
                    messages=[
                        {"role": "user", "content": user_text},
                        {"role": "assistant", "content": first.content},
                        {"role": "user", "content": [
                            {"type": "tool_result", "tool_use_id": part.id, "content": json.dumps(result)}
                        ]},
                    ],
                )

                for c in getattr(follow, "content", []):
                    if hasattr(c, "text") and c.text:
                        output_text += c.text + "\n"

            elif hasattr(part, "text") and part.text:
                output_text += part.text + "\n"
    #exit loop through user input
        if output_text.strip():
            print(output_text.strip())
        if user_text.lower() in ("quit"):
            break

Welcome to the Messier Object Tourist Guide!
You can ask for a variety of information all about Messier Objects such as their type, magnitude, best viewing time, etc.
Quit → stop the program




>  What is m33


Let me retrieve the story for M33 for you.
M33, also known as the Triangulum Galaxy, is a spiral galaxy located in the constellation Triangulum. It's a relatively bright galaxy with a magnitude of 6.3, which means it can be seen with binoculars or a small telescope under good viewing conditions. It's classified as an Sc type spiral galaxy, often referred to as a "Pinwheel Galaxy" due to its distinctive spiral structure. M33 is part of the Local Group of galaxies, which includes the Milky Way and Andromeda Galaxy, and is one of the group's smaller members.

Would you like to know more about this fascinating galaxy?



>  no


I apologize, but your input doesn't seem to be a clear request or question about the Messier catalog. Would you like to ask something about Messier objects, astronomical objects, or celestial observations?



>  quit


Goodbye


Full work breakdown. Dilmar mainly worked on the LLM functionality and setup while Torsten focused on creating the function tools to be used by the LLM. Both worked on debugging. Dilmar also helped out with some of the function tools specifically around the object story. 

Findings and Observations: I found that we could have improved on the layout and have made a GUI for the object guide. Could have included more functions or a skyfield but time did not permit it. 